In [1]:
import cv2
import numpy as np
import cvlib as cv

from collections import deque
from timeit import default_timer as timer
from cvlib.object_detection import draw_bbox

# Catch frame from webcam
camera = cv2.VideoCapture(1)

# Define variables for hight and width shape of the frames
HEIGH, WIDTH = 400, 500   

# Scale to decrease the frame size
SCALE = 1

# Variable define timer of checking tracking status
CHECKER = 10

# Initializing deque object for center points of the detected object
points = deque(maxlen=50)

# Restart timer for FPS
fps_start = timer()    
    
# Increasing FPS counter
counter_fps = 0

# Define the background
last_frame = np.zeros((int(HEIGH/SCALE), int(WIDTH/SCALE), 3) , np.uint8)

# Variable counting how many time we are tracking after the objects
frames_tracking_counter = 0

# Variable counting how many time we are trying to detect objects
counter_frames_predictions = 0

# Variable counting how many object we detected in every iteration
object_detected_counter = 0

# Variable counting how many times we read frames
frames_reading_counter = 0

# Variable store the system status of tracking or not tracking
tracking_on = False

# Define the tresh hold of the masks
DIFF_TRESH_HOLD = 20  # Should be low
MASK_TRESH_HOLD = 100 # Should be high

# Define tracker dictionary
tracker_dict = { 'csrt': cv2.legacy.TrackerCSRT_create,
                 'kcf' : cv2.legacy.TrackerKCF_create,
                 'boosting' : cv2.legacy.TrackerMOSSE_create,
                 'mil': cv2.legacy.TrackerMIL_create,
                 'tld': cv2.legacy.TrackerTLD_create,
                 'medianflow': cv2.legacy.TrackerMedianFlow_create,
                 'mosse':cv2.legacy.TrackerMOSSE_create}

########################################################################################################################
    
# Function return 3-Dimension frame
def expands_dimensions(frame):
    
    new_image = np.zeros((frame.shape[0], frame.shape[1], 3), np.uint8)
    new_image[:, :, 0] = frame
    new_image[:, :, 1] = frame
    new_image[:, :, 2] = frame
    
    return new_image

# Convert frame from rgb to gray
def gray_frame(frame_rgb):
    
    # Converting captured frame to GRAY by OpenCV function    
    gray_frame = cv2.cvtColor(frame_rgb, cv2.COLOR_RGB2GRAY)
    
    frame_gray = np.zeros(frame_rgb.shape, np.uint8)
    frame_gray[:,:,0] = gray_frame
    frame_gray[:,:,1] = gray_frame
    frame_gray[:,:,2] = gray_frame
    
    return frame_gray

# Function define the mask
def mask(frame_rgb, background):
    
    # Converting captured frame to GRAY by OpenCV function    
    frame_gray = cv2.cvtColor(frame_rgb, cv2.COLOR_RGB2GRAY)
    
    # Create one more frame with Gaussian blur
    frame_gray = cv2.GaussianBlur(frame_gray, (25, 25), 0)  

    # Converting captured frame to GRAY by OpenCV function        
    background = cv2.cvtColor(background, cv2.COLOR_BGR2GRAY)
    
    # Create one more frame with Gaussian blur
    background = cv2.GaussianBlur(background, (25, 25), 0)    
    
    # Return mask to detect change between two frames   
    abs_diff = cv2.absdiff(frame_gray, background)
    
    # Function exclude values that ara more than treshhold = 15 0 and more than 255
    _, mask = cv2.threshold(abs_diff, 20, 255, cv2.THRESH_BINARY)
    
    dilated_mask = cv2.dilate(mask, None, iterations = 5)

    # Expend mask dimension to 3 dimension
    mask_frame = expands_dimensions(dilated_mask)        

    return mask_frame

def mask_tracking(frame_RGB, last_frame):
    
    # Converting captured frame to GRAY by OpenCV function    
    frame_gray = cv2.cvtColor(frame_RGB, cv2.COLOR_RGB2GRAY)
    
    # Converting captured frame to GRAY by OpenCV function        
    last_frame = cv2.cvtColor(last_frame, cv2.COLOR_BGR2GRAY)
     
    # Return mask to detect change between two frames   
    abs_diff = cv2.absdiff(frame_gray, last_frame)
    
    # Function exclude values that ara more than treshhold = 15 0 and more than 255
    _, abs_diff_mask = cv2.threshold(abs_diff, DIFF_TRESH_HOLD, 255, cv2.THRESH_BINARY)

    # Expend mask dimension to 3 dimension
    mask_frame = expands_dimensions(abs_diff_mask)        
    
    return mask_frame

# Function manage the frames reader variables like efps etc'
def reader_manger(fps_start, counter_fps, tracking_on):
    
    # Variable says if keep reading frame or quit
    quit = False

    # Stopping the timer for FPS
    fps_stop = timer()

    # Print FPS every 1 second
    if 1.0 <= fps_stop - fps_start:

        # Define FPS
        FPS = counter_fps

        # Reset FPS counter
        counter_fps = 0

        # Restart timer for FPS
        fps_start = timer()       

    # Function waits for key to be pressed    
    key = cv2.waitKey(1) % 256

    # If 'n' is pressed, we catchs the frame and define it as the background
    if key == ord('n'):
        tracking_on = False

    # If 'q' key is pressed then quit from app
    if key == ord('q'):
        quit = True   

    return FPS, counter_fps, tracking_on, quit

# Function display the frames on the screen in one window
def display_windows(frame, tracking_mask, frame_mask):   
        
    # Create left window    
    main_window = np.hstack((frame, tracking_mask, frame_mask))

    # Plotting all the frames in one window
    cv2.imshow("Main_Window", main_window)     
    
# Function create 3 frames from the frame we read
def preproccess_frames(frame, last_frame):
    
    # Define small sizes
    heigh, width = int(HEIGH/SCALE), int(WIDTH/SCALE)
    
    # Resize the main frame to (WIDTH, HEIGH) shape
    frame = cv2.resize(frame, (WIDTH, HEIGH))
        
    # Copy frame to work with deffrent variable
    frame_rgb = cv2.resize(frame, (width, heigh))
    
    # Return mask for detection 
    frame_mask = mask(frame_rgb, last_frame)
    
    # Return mask for tracking
    tracking_mask = mask_tracking(frame_rgb, last_frame)
    
    # Define last frame
    last_frame = frame_rgb.copy()
    
    return frame, frame_rgb, tracking_mask, frame_mask, last_frame
    
# Function manage the detection and return status and coordinates
def detection_manager(frame, frame_rgb):

    # Store the bounding boxes with the new coordinates in a list
    boxes = []
    
    # Variable count how many objects we detect
    num_of_objects = 0

    # Function return all scores of model predictions
    bounding_boxes, detected_labels, scores = cv.detect_common_objects(frame_rgb)

    # Check if we succeeded to detect objects
    if 0 < len(bounding_boxes):
        
        # Scaling the bounding boxes back to original main frame size
        for box in bounding_boxes:

            # Increase the objects counter
            num_of_objects += 1

            # Create new list of bounding boxes that fit to main frame size
            (x_min, y_min, x_max, y_max) = [int(a) for a in box]
            
            # bounding_boxes contain x1, y1, x2, y2, coordinates and not width and heigh
            bounding_box = np.array([x_min*SCALE, y_min*SCALE, x_max*SCALE, y_max*SCALE])
            boxes.append(bounding_box)

            # Value means we start tracking after the objects
            tracking_status = "Start tracking"
    else:      
        # Value means no tracking need and have to try detect again
        tracking_status = "End tracking"
            
    return boxes, detected_labels, scores, num_of_objects, tracking_status

# Function manages the start tracking case
def start_tracking(frame, bounding_boxes):
    
    # Initialize our tracker after the object
    trackers = cv2.legacy.MultiTracker_create()
    
    # Update boxes list to the original main frame size scale
    for box in bounding_boxes:

        # Create rectangle that use us to tracking after the object and fit to main frame size
        (x_min, y_min, x_max, y_max) = [int(a) for a in box]
        
        # rectangle object contain x1, y1, box width, box height and not x,y max coordinates
        rectangle = np.array([x_min*SCALE, y_min*SCALE, (x_max-x_min)*SCALE, (y_max-y_min)*SCALE])
        
        # Add the object to the trackers list
        tracker_i = tracker_dict['csrt']()
        trackers.add(tracker_i, frame, rectangle)

    return trackers    

# Function manage the tracking and return status and coordinates
def tracking_manager(frame, tracking_mask, bounding_boxes, tracking_status, frames_tracking_counter):
       
    # Check if there is still object to track after
    if tracking_status == 'End Tracking':
        
        return frame, tracking_status, trackers
            
    # Check if we are just start the tracking or we are just keeping it
    if tracking_status == 'Start tracking':
          
        trackers = start_tracking(frame, bounding_boxes)      
        return frame, tracking_status, trackers
    
#     # We Keep the previous tracking        
#     if tracking_status == 'Keep tracking':
        
#         frame = keep_tracking(frame, bounding_boxes)
#         return frame, tracking_status, _
        
    return frame, trackers, tracking_status

# Function manges the keep tracking case
def keep_tracking_manager(frame, trackers, frames_tracking_counter, tracking_mask, detected_labels, tracking_status, tracking_on):
    
    # Get the bounding box from the frame
    (success, bounding_boxes) = trackers.update(frame)  

    # Strart\Keep tracking
    if success:      

        # Variable is index of box in the bounding_boxes
        index = 0 

        for box in (bounding_boxes):        

            # Get the coordinates of the rectangle around the object
            (x, y, w, h) = [int(a) for a in box]

            # Check if coordinates is in the frame boundaries
            if 0 < x and x+w < WIDTH and 0 < y and y+h < HEIGH:  

                # Every 25 frames checking if tracking is still running after the object or not
                if(frames_tracking_counter%15 == 0):  

                    # Checking onlt for birds
                    if(detected_labels[index] == "bird"):

                        # Cut the fragment from the mask frame
                        cut_fragment_mask = tracking_mask[y:y+h, x:x+w]

                        # Cheking the status by sum the pixels values in the rectangle
                        if np.sum(cut_fragment_mask) == 0:

                            #bounding_boxes = np.delete(bounding_boxes, index, 0)

                            # Update tracking status
                            tracking_status = "End tracking"

                            # Initializes variable
                            tracking_on = False 
                            break

                # Set tracking status ON
                tracking_on = True       

                # Update tracking status
                tracking_status = "Keep tracking"              

                # Drawing bounding box on the current BGR frame        
                cv2.rectangle(frame, (x,y), (x+w,y+h), (100,255,0), 2)

                # Putting text with label on the current BGR frame
                cv2.putText(frame, detected_labels[index], (x-5, y-5), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0, 255, 0), 2)  

            # Increase index by 1
            index += 1
            
    return frame, tracking_status, tracking_on, trackers



In [2]:
# Loop reading frame by frame and processing them
while True:
    
    # Increasing FPS counter
    counter_fps += 1
    
    # Capturing frames one-by-one from camera
    ret, frame = camera.read()

    # If the frame was not retrieved then we break the loop
    if not ret or frame is None:
        break
        
    # Increase tracking counter
    frames_reading_counter += 1
        
    # Function return 3 diffrent kind of frames
    frame, frame_rgb, tracking_mask, frame_mask, last_frame = preproccess_frames(frame, last_frame)
    
    # Treats objects tracking
    if tracking_on == True:
        
        # Increase tracking counter
        frames_tracking_counter += 1

        # Function manage the the part of the tracking   
        frame, tracking_status, tracking_on, trackers = keep_tracking_manager(frame, trackers, frames_tracking_counter, tracking_mask,
                                                                         detected_labels, tracking_status, tracking_on)
                        
        # Update tracking status for the next iterate
        if tracking_status == "End tracking":
            tracking_on = False 
        if tracking_status == "Keep tracking":
            tracking_on = True
        
    # End of tracking_on - Treats objects detection
    else:

        # Increase prediction counter
        counter_frames_predictions += 1
        
        # Function manage the detection part and return coordinates of drawing   
        bounding_boxes, detected_labels, scores, object_detected_counter, tracking_status = detection_manager(frame, frame_rgb)
        
        # Function draw boxes around the detected objects
        frame = draw_bbox(frame, bounding_boxes, detected_labels, scores)
        
        # Function manage the first part of the tracking and return coordinates of tracking    
        frame, tracking_status, trackers = tracking_manager(frame, last_frame, bounding_boxes, 
                                                            tracking_status, frames_tracking_counter)
        
        # Update tracking status to decide what next in the next iteration
        if tracking_status == "Start tracking":
            tracking_on = True
        elif tracking_status == "End tracking":
            tracking_on = False
        
        
    # Display all frames in one window
    display_windows(frame, tracking_mask, frame_mask)
       
    # Restets the objects counter every iteration of frame reading 
    num_of_objects = len(bounding_boxes)

    # Function manage the frames reader variables    
    FPS, counter_fps, tracking_on, quit = reader_manger(fps_start, counter_fps, tracking_on)

    # If quit is true so we stop read frames    
    if quit == True:
        break

# Releasing camera
camera.release()

# Destroying all opened OpenCV windows
cv2.destroyAllWindows()      